<a href="https://colab.research.google.com/github/helpjulia/03012024_data_temoa_energy_modeling/blob/master/random.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install -q pyomo
from pyomo.environ import *

#create a model
model = ConcreteModel()

#create decision variables
model.x = Var(within=NonNegativeReals)
model.y = Var(within=NonNegativeReals)

#create model objective
model.z = Objective(expr = 3*model.x + 5*model.y, sense=maximize)

#declare constraints
model.c1 = Constraint(expr = model.x <= 4)
model.c2 = Constraint(expr = 2*model.y <= 12)
model.c3 = Constraint(expr = 3*model.x + 2*model.y <= 18)

model.pprint()

2 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    y : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    z : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 3*x + 5*y

3 Constraint Declarations
    c1 : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :  -Inf :    x :   4.0 :   True
    c2 : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :  -Inf :  2*y :  12.0 :   True
    c3 : Size=1, Index=None, Active=True
        Key  : Lower : Body      : Upper : Active
        None :  -Inf : 3*x + 2*y :  18.0 :   True

6 Declarations: x y z c1 c2 c3


In [9]:
!apt-get install -y -qq glpk-utils

In [10]:
SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model).write()

#display solution
print('\nProfit = ', model.z()) # Access the objective value using model.z()
print('.\nDecision Variables')
print('x = ', model.x())
print('y = ', model.y())

#display constraints
print('\nConstraints')

#print('Demand = ', model.demand())
print('Constraint 1 = ', model.c1())
print('Constraint 2 = ', model.c2())
print('Constraint 3 = ', model.c3())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 36.0
  Upper bound: 36.0
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 2
  Number of nonzeros: 4
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.0058324337005615234
# ----------------------------------------------------------
#   Solution Information
# -------------------------------------

In [17]:
from pyomo.environ import *
infinity = float('inf')

model = ConcreteModel()

# Factories
model.I = Set(initialize=["F_A", "F_B"])

# Pollutants
model.J = Set(initialize=["P_1", "P_2"])

# ci - Cost to process waste from factory
model.c = Param(model.I, model.J, initialize={"F_A": 15, "F_B": 10})

# rij - Rate of reduction in pollutant type if emission is treated at factory
model.r = Param(model.I, model.J, initialize={("F_A",'P_1'):0.1,("F_A",'P_2'):0.4,("F_B",'P_1'):0.2, ("F_B",'P_2'):0.16})
# Lower and upper bound on processed waste
model.Wmin = Param(model.I, within=NonNegativeReals, default=0.0)
# model.Wmax = Param(model.I, within=NonNegativeReals, default=infinity)
# Sj - State requirement to reduce pollutant
model.S = Param(model.J, initialize={'P_1':30, 'P_2':40})
# Minimize the cost to reduce pollutants
def cost_rule(model):
    return sum(model.c[i]*model.w[i] for i in model.I)
model.cost = Objective(rule=cost_rule)

# Satisfy the state's requirements on pollutant reduction
def pollutant_rule(model, j):
    return sum(model.r[i,j]*model.w[i] for i in model.I) >= model.S[j]
model.pollutant = Constraint(model.J, rule=pollutant_rule)

# Limit the volume of the decision variables
def volume_rule(model, i):
    return model.w[i] >= model.Wmin[i]
model.volume = Constraint(model.I, rule=waste_rule_min)


ERROR:pyomo.core:Rule failed for Param 'c' with index F_A:
KeyError: "Index 'F_A' is not valid for indexed component 'c'"
ERROR:pyomo.core:Constructing component 'c' from data=None failed:
    KeyError: "Index 'F_A' is not valid for indexed component 'c'"


KeyError: "Index 'F_A' is not valid for indexed component 'c'"

In [32]:
from pyomo.environ import *
infinity = float('inf')

model = ConcreteModel()

# Factories
model.I = Set(initialize=["F_A", "F_B"])

# Pollutants
model.J = Set(initialize=["P_1", "P_2"])

# ci - Cost to process waste from factory
# Provide values for all combinations of factories and pollutants
model.c = Param(model.I, model.J, initialize={
    ("F_A", "P_1"): 15,
    ("F_B", "P_1"): 10,
    ("F_A", "P_2"): 20,  # Add a value for F_A and P_2
    ("F_B", "P_2"): 12   # Add a value for F_B and P_2
})

# rij - Rate of reduction in pollutant type if emission is treated at factory
model.r = Param(model.I, model.J, initialize={("F_A",'P_1'):0.1,("F_A",'P_2'):0.4,("F_B",'P_1'):0.2, ("F_B",'P_2'):0.16})
# Lower and upper bound on processed waste
model.Wmin = Param(model.I, within=NonNegativeReals, default=0.0)
# model.Wmax = Param(model.I, within=NonNegativeReals, default=infinity)
# Sj - State requirement to reduce pollutant
model.S = Param(model.J, initialize={'P_1':30, 'P_2':40})

# Decision variable for the amount of waste processed
model.w = Var(model.I, within=NonNegativeReals)

# Minimize the cost to reduce pollutants
def cost_rule(model):
    # Iterate over both factories and pollutants
    return sum(model.c[i, j] * model.w[i] for i in model.I for j in model.J)
model.cost = Objective(rule=cost_rule)

# Satisfy the state's requirements on pollutant reduction
def pollutant_rule(model, j):
    return sum(model.r[i,j]*model.w[i] for i in model.I) >= model.S[j]
model.pollutant = Constraint(model.J, rule=pollutant_rule)

# Limit the volume of the decision variables
def volume_rule(model, i):
    return model.w[i] >= model.Wmin[i]
# Fix typo in constraint name
model.volume = Constraint(model.I, rule=waste_rule_min)

SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model).write()
print("\nCost = ", model.cost())
for i in model.I:
    print(i, model.w[i]())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 4500.0
  Upper bound: 4500.0
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 2
  Number of nonzeros: 6
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.004494667053222656
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------

In [26]:
from pyomo.environ import *
infinity = float('inf')

model = AbstractModel()

# Factories
model.I = Set()

# Pollutants
model.J = Set()

# ci - Cost to process waste from factory
model.c = Param(model.I, within=PositiveReals)

# rij - Rate of reduction in pollutant type if emission is treated at factory
model.r = Param(model.I, model.J, within=NonNegativeReals)
# Lower and upper bound on processed waste
model.Wmin = Param(model.I, within=NonNegativeReals, default=0.0)
model.Wmax = Param(model.I, within=NonNegativeReals, default=infinity)
# Sj - State requirement to reduce pollutant
model.S = Param(model.J)

# wi - Processed waste from Factory
model.w = Var(model.I, within=NonNegativeReals)

# minimize the cost to reduce pollutants
def cost_rule(model):
    return sum(model.c[i]*model.w[i] for i in model.I)
model.cost = Objective(rule=cost_rule)

# satisfy the state's requirements on pollutant reduction
def pollutant_rule(model, j):
    return sum(model.r[i,j]*model.w[i] for i in model.I) >= model.S[j]
model.pollutant = Constraint(model.J, rule=pollutant_rule)

# limit the volume of the decision variables
def waste_rule_min(model, i):
    return model.w[i] >= model.Wmin[i]
model.volume = Constraint(model.I, rule=waste_rule_min)

SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model).write()
print("\nCost = ", model.cost())
for i in model.I:
    print(i, model.w[i]())

RuntimeError: Attempting to solve model=unknown with unconstructed component(s)

In [ ]:
from itertools import tee
from pyomo.environ import *
from riverpol import model
instance = model.create_instance('riverpol.dat')
instance.pprint()
opt = SolverFactory('cplex')
results = opt.solve(instance, tee=True)
instance.write('problem.lp')

print(instance.cost())


In [16]:
!pip install -q pyomo
from math import inf
from pyomo.environ import*


model = AbstractModel()

model.I = Set() # Set of Energy Production
model.J = Set() # Set of Fuels

# Parameters Definition
# ri - revenue for producing/selling energy type i [$/m3]
model.r    = Param(model.I, within=PositiveReals)
# qij - Fuel type j used for energy production type i [kg/m3]
model.q    = Param(model.I, model.J, within=NonNegativeReals)
# bj - max available stock of fuel type j
model.b = Param(model.J,within=NonNegativeReals)

# gi - necessary proportions of energy type i
model.g = Param(model.I,within=NonNegativeReals)

# Decision Variables Definition
model.x = Var(model.I, within=NonNegativeReals) # xi - amount of energy type i to produce

# Objective Function - Maximize the cost of waste treatment to reduce pollutants
def cost_rule(model):
    return sum(model.r[i]*model.x[i] for i in model.I)
model.cost = Objective(rule=cost_rule,sense=maximize)

# Constraints - Satisfy max available fuel type j volume
def fuel_rule(model, j):
    return sum(model.q[i,j]*model.x[i] for i in model.I) <= model.b[j]
model.fueltype_constraints = Constraint(model.J, rule=fuel_rule)

def prop_rule(model):
	return sum(model.g[i]*model.x[i] for i in model.I) >= 0
model.proprule = Constraint(rule=prop_rule)


In [15]:
!apt-get install -y glpk-utils
import os
os.environ['PATH'] += ':/usr/bin'  # Add the directory containing glpk to PATH
# Restart the kernel here. You can do this manually or by running the following in a code cell:
# !kill -9 -1
instance = model.create_instance('biogas.dat')
instance.pprint()
opt = SolverFactory('glpk')
results = opt.solve(instance, tee=True)
print(instance.display())

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
glpk-utils is already the newest version (5.0-1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


2 Set Declarations
    I : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'biomass', 'biomethane'}
    J : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'horsedrop', 'liqmanure'}

4 Param Declarations
    b : Size=2, Index=J, Domain=NonNegativeReals, Default=None, Mutable=False
        Key       : Value
        horsedrop : 126.0
        liqmanure :  84.0
    g : Size=2, Index=I, Domain=NonNegativeReals, Default=None, Mutable=False
        Key        : Value
           biomass :   2.0
        biomethane :   1.0
    q : Size=4, Index=I*J, Domain=NonNegativeReals, Default=None, Mutable=False
        Key                         : Value
           ('biomass', 'horsedrop') :   8.0
           ('biomass', 'liqmanure') :   2.0
        ('biomethane', 'horsedrop') :   3.0
        ('biomethane', 'liqmanure') :   6.0
    r : Size=2, Index=I, Do

ApplicationError: No executable found for solver 'glpk'

In [17]:
!apt-get install -y glpk-utils
instance = model.create_instance('biogas.dat')
instance.pprint()
opt = SolverFactory('glpk')
results = opt.solve(instance, tee=True)
print(instance.display())

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
glpk-utils is already the newest version (5.0-1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


2 Set Declarations
    I : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'biomass', 'biomethane'}
    J : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'horsedrop', 'liqmanure'}

4 Param Declarations
    b : Size=2, Index=J, Domain=NonNegativeReals, Default=None, Mutable=False
        Key       : Value
        horsedrop : 126.0
        liqmanure :  84.0
    g : Size=2, Index=I, Domain=NonNegativeReals, Default=None, Mutable=False
        Key        : Value
           biomass :   2.0
        biomethane :   1.0
    q : Size=4, Index=I*J, Domain=NonNegativeReals, Default=None, Mutable=False
        Key                         : Value
           ('biomass', 'horsedrop') :   8.0
           ('biomass', 'liqmanure') :   2.0
        ('biomethane', 'horsedrop') :   3.0
        ('biomethane', 'liqmanure') :   6.0
    r : Size=2, Index=I, Do

ApplicationError: No executable found for solver 'glpk'